In [1]:
# High level design
# - User inputs multiple hands and a partial board, gets back probabilities from MC 
# - Checks for duplicate cards
# - Encode cards and suits into mod 36
# - Searches for hand ranks, no pruning, returns strength
# - Comparator for hands within same rank
# - Simulation over trials to get probabilities
import random
from timeit import default_timer as timer
import numpy as np
import itertools as it

In [2]:
# build global vars
suits = ["c", "d", "h", "s"]
cards = [str(x) for x in range(6, 15)]
cards = [x.replace('10', 'T').replace('11', 'J').replace('12', 'Q').replace('13', 'K').replace('14', 'A') for x in cards]
deck = [card + suit for card in cards for suit in suits]
value_map = {deck[i]: i for i in range(len(deck))}
card_map = {v: k for k, v in value_map.items()}

In [3]:
def check_dumbassery(hands, board):
    in_play = [card for hand in hands for card in hand] + board
    unique = set(in_play)
    if (len(unique) < len(in_play)):
        print("duplicate cards")
        print(set([card for card in in_play if in_play.count(card) > 1]))

In [4]:
# 0 - club, 1 - diamond, 2 - heart, 3 - spade
def get_suit(value):
    return value%4

# 0 - 6 up to 8 - Ace
def get_face(value):
    return value//4

In [5]:
def evaluate_hands(hands, board, debug=False):
    mx_rank = 0
    mx_hands = []
    for hand in hands:
        new = evaluate_combo(hand+board)
        if (new > mx_rank):
            mx_rank = new
            mx_hands = [hand]
        elif (new == mx_rank):
            mx_hands.append(hand)
    if (debug):
        print(hand_rank[mx_rank])
    if (len(mx_hands) == 1):
        return mx_hands[0]
    else:
        return compare_same_rank(hands, board, mx_rank)

In [6]:
def quantify(res):
    if (isinstance(res, int)):
        return res
    elif res is None:
        return -1
    else:
        return int(''.join(map(str, res)))

In [7]:
def compare_same_rank(hands, board, rank):
    f = [quantify(get_hand[rank](hands[i]+board)) for i in range(len(hands))]
    res = [hands[i] for i in range(len(f)) if f[i] == max(f)]
    if (len(res) == 1):
        return res[0]
    else:
        return res

In [8]:
# straight flush - 8
# quads - 7
# flush - 6
# full house - 5
# straight - 4
# trips - 3
# two pair - 2
# pair - 1
# high card - 0
def evaluate_combo(combo):
    res = get_straight_flush(combo)
    if res > 0:
        return 8 
    res = get_quads(combo)
    if res is not None:
        return 7
    res = get_flush(combo)
    if res is not None:
        return 6
    res = get_full_house(combo)
    if res is not None:
        return 5
    res = get_straight(combo)
    if res > 0:
        return 4
    res = get_trips(combo)
    if res is not None:
        return 3
    res = get_two_pair(combo)
    if res is not None:
        return 2
    res = get_pair(combo)
    if res is not None:
        return 1
    return 0

In [9]:
def get_straight_flush(combo): # 6 rankings of straight flushes 
    s = is_flush(combo)
    if s is not None:
        h = [x for x in combo if get_suit(value_map[x]) == s]
        res = get_straight(h)
        if res > 0:
            return res
    return -1

In [10]:
def get_quads(combo):
    f = [get_face(value_map[x]) for x in combo]
    d = {i:f.count(i) for i in set(f)}
    h = max(d.keys(), key=(lambda key: d[key])) # can only be one set of quads
    if (d[h] < 4):
        return None
    rem = [x for x in f if x != h]
    mx = 0 
    for card in rem:
        if (card > mx):
            mx = card
    res = [h]
    res.append(mx)
    return res

In [11]:
def get_flush(combo): # assumes straight flush already filtered 
    s = is_flush(combo)
    if s is not None:
        fc = [get_face(value_map[x]) for x in combo if get_suit(value_map[x]) == s]
        fc.sort(reverse=True)
        return fc[0:5]
    else:
        return None

In [12]:
def is_flush(combo):
    s = [get_suit(value_map[x]) for x in combo]
    l = [s.count(x) for x in range(4)]
    if (len([x for x in l if x >= 5]) == 1):
        suit = l.index(max(l))
        return suit
    return None

In [13]:
def get_full_house(combo): # assumes quads already filtered
    f = [get_face(value_map[x]) for x in combo]
    d = {i:f.count(i) for i in set(f)}
    h = max(d.keys(), key=(lambda key: d[key])) 
    h2 = max(d.keys(), key=(lambda key: d[key] if key != h else -1))
    h3 = max(d.keys(), key=(lambda key: d[key] if (key != h) & (key != h2) else -1))
    if (d[h] != 3):
        return None
    else:
        if (d[h2] < 2):
            return None
        elif (d[h2] == 2):
            if (d[h3] == 2):
                if (h2 < h3):
                    return [h, h3]
                else:
                    return [h, h2]
            else:
                return [h, h2]
        elif (d[h2] == 3): # can only be 2 sets of trips     
            if (h < h2):
                return [h2, h]
            else:
                return [h, h2]
        else:
            print("error")

In [14]:
def get_straight(combo):
    f = [get_face(value_map[x]) for x in combo]
    if 4 in f:
        if 5 in f:
            if 6 in f:
                if 7 in f:
                    if 8 in f:
                        return 6
                    elif 3 in f:
                        return 5
                elif 3 in f:
                    if 2 in f:
                        return 4
            elif 3 in f:
                if 2 in f:
                    if 1 in f:
                        return 3
        elif 3 in f:
            if 2 in f:
                if 1 in f:
                    if 0 in f:
                        return 2
    elif 3 in f:
        if 2 in f:
            if 1 in f:
                if 0 in f:
                    if 8 in f:
                        return 1
    return 0

In [15]:
def get_trips(combo): # assumes quads, full house already filtered
    f = [get_face(value_map[x]) for x in combo]
    d = {i:f.count(i) for i in set(f)}
    h = max(d.keys(), key=(lambda key: d[key])) 
    if (d[h] != 3):
        return None
    rem = [x for x in f if x != h]
    rem.sort(reverse=True)
    res = [h] + rem[0:2]
    return res

In [16]:
def get_two_pair(combo): # assumes trips and above filtered
    f = [get_face(value_map[x]) for x in combo]
    d = {i:f.count(i) for i in set(f)}
    h = max(d.keys(), key=(lambda key: d[key])) 
    h2 = max(d.keys(), key=(lambda key: d[key] if key != h else -1))
    h3 = max(d.keys(), key=(lambda key: d[key] if ((key != h) & (key != h2)) else -1))
    if (d[h] != 2):
        return None
    else:
        if (d[h2] != 2):
            return None
        else:
            if (d[h3] == 2):
                l = [h, h2, h3]
                l.sort(reverse=True)
                rem = max([x for x in f if x not in l[0:2]])
                return l[0:2] + [rem]
            else:
                l = [h, h2]
                l.sort(reverse=True)
                rem = max([x for x in f if x not in l])
                return l + [rem]

In [17]:
def get_pair(combo): 
    f = [get_face(value_map[x]) for x in combo]
    d = {i:f.count(i) for i in set(f)}
    h = max(d.keys(), key=(lambda key: d[key]))
    if (d[h] != 2):
        return None
    rem = [x for x in f if x != h]
    rem.sort(reverse=True)
    res = [h] + rem[0:3]
    return res

In [18]:
def get_high_card(combo):
    f = [get_face(value_map[x]) for x in combo]
    f.sort(reverse=True)
    return f[0:5]

In [19]:
# test cases
p0 = ["As", "8s"]
p1 = ["9s", "9c"]
board = ["9s", "Qc", "Jc", "7s", "6s"]
print(get_straight_flush(p0 + board))
print(get_flush(p0+board))
board = ["8c", "9c", "8d", "8s", "Td"]
print(get_quads(p0 + board))
board = ["8c", "9c", "8d", "9s", "Td"]
print(get_full_house(p0+board))
board = ["8c", "Ac", "8d", "Ad", "Td"]
print(get_full_house(p0+board))
board = ["9d", "8c", "8d", "Ts", "Tc"]
print(get_full_house(p1+board))
board = ["6c", "Kd", "7c", "9s", "Td"]
print(get_straight(p0+board))
board = ["Qc", "Kd", "Jc", "9s", "Td"]
print(get_straight(p0+board))
board = ["9d", "Qs", "8c", "6d", "7s"]
print(get_trips(p1+board))
board = ["Ad", "Kd", "Ts", "Ac", "6d"]
print(get_trips(p0+board))
board = ["Ac", "8c", "8d", "As", "Kd"]
print(get_two_pair(p1+board))
board = ["Ac", "8c", "9d", "9s", "Kd"]
print(get_two_pair(p0+board))
board = ["8d", "Ks", "Jd", "Qh", "9c"]
print(get_pair(p0+board))
board = ["Ks", "Jd", "7h", "6d", "8s"]
print(get_pair(p1+board))
board = ["Ks", "Qd", "Js", "7d", "6s"]
print(get_high_card(p0+board))

1
[8, 3, 2, 1, 0]
[2, 8]
[2, 3]
[8, 2]
[3, 4]
2
6
[3, 6, 2]
[8, 7, 4]
[8, 3, 7]
[8, 3, 7]
[2, 8, 7, 6]
[3, 7, 5, 2]
[8, 7, 6, 5, 2]


In [20]:
get_hand = {8:get_straight_flush, 
            7:get_quads,
            6:get_flush,
            5:get_full_house,
            4:get_straight,
            3:get_trips,
            2:get_two_pair,
            1:get_pair,
            0:get_high_card}

In [21]:
hand_rank = {8:"straight flush", 
             7:"quads",
             6:"flush",
             5:"full house",
             4:"straight",
             3:"trips",
             2:"two pair",
             1:"pair",
             0:"high card"}

In [22]:
def simulate(hands, board, trials=10000, debug=False):
    cards = [card for hand in hands for card in hand] + board
    revealed = [value_map[card] for card in cards]
    deck = [x for x in range(36) if x not in revealed]
    tallies = dict.fromkeys([tuple(hand) for hand in hands], 0)
    for i in range(trials):
        draw = random.sample(deck, 5-len(board))
        runout = [card_map[card] for card in draw]
        full_board = board + runout
        res = evaluate_hands(hands, full_board, debug)
        if (debug):
            print(full_board, res)
        if(any(isinstance(i, list) for i in res)):
            inc = 1/len(res)
            for hand in res:
                tallies[tuple(hand)] = tallies[tuple(hand)] + inc
        else:
            tallies[tuple(res)] = tallies[tuple(res)] + 1
    for hand in hands:
        tallies[tuple(hand)] = tallies[tuple(hand)]/trials

    for k, v in tallies.items():
        tallies[k] = round(v, 3)

    print(tallies)

In [28]:
# hole cards
p1 = ["As","Th"] 
p2 = ["Tc", "8d"]
p3 = ["Ad", "Ah"]
hands = [p1, p2, p3]
board = ["6d", "9h", "Ac"]
check_dumbassery(hands, board)

In [29]:
start = timer()
simulate(hands, board, trials=10000, debug=False)
end = timer()
print(end-start)

{('As', 'Th'): 0.019, ('Tc', '8d'): 0.22, ('Ad', 'Ah'): 0.761}
2.4822385445577737


In [25]:
board = ["Qs", "Ks", "Js", "Kd", "Th"]
evaluate_hands(hands, board, debug=True)

straight


[['As', 'Th'], ['Ad', 'Ah']]

In [26]:
# create numerical range table, + -> suited, -> offsuit
primes = [2, 3, 5, 7, 11, 13, 17, 19, 23]
face_ranks = list(reversed(primes))
cards_rev = list(reversed(cards))
face_map = dict(zip(cards_rev, face_ranks))
hands_map = {}
for x in it.product(cards_rev, cards_rev):
    if (x[0] == x[1]):
        hands_map[x[0]+x[1]] = -face_map[x[0]]**2
    if (face_map[x[0]] > face_map[x[1]]):
        hands_map[x[0]+x[1]+"s"] = face_map[x[0]]*face_map[x[1]]
        hands_map[x[0]+x[1]+"o"] = -face_map[x[0]]*face_map[x[1]]
to_hands_map = {v: k for k, v in hands_map.items()}
value_table = np.multiply(np.outer(face_ranks, face_ranks), np.tril(-2*np.ones((9, 9)), 0)+1)
hands_table = [[to_hands_map[x] for x in row] for row in value_table.astype(int).tolist()]

In [27]:
hands_table

[['AA', 'AKs', 'AQs', 'AJs', 'ATs', 'A9s', 'A8s', 'A7s', 'A6s'],
 ['AKo', 'KK', 'KQs', 'KJs', 'KTs', 'K9s', 'K8s', 'K7s', 'K6s'],
 ['AQo', 'KQo', 'QQ', 'QJs', 'QTs', 'Q9s', 'Q8s', 'Q7s', 'Q6s'],
 ['AJo', 'KJo', 'QJo', 'JJ', 'JTs', 'J9s', 'J8s', 'J7s', 'J6s'],
 ['ATo', 'KTo', 'QTo', 'JTo', 'TT', 'T9s', 'T8s', 'T7s', 'T6s'],
 ['A9o', 'K9o', 'Q9o', 'J9o', 'T9o', '99', '98s', '97s', '96s'],
 ['A8o', 'K8o', 'Q8o', 'J8o', 'T8o', '98o', '88', '87s', '86s'],
 ['A7o', 'K7o', 'Q7o', 'J7o', 'T7o', '97o', '87o', '77', '76s'],
 ['A6o', 'K6o', 'Q6o', 'J6o', 'T6o', '96o', '86o', '76o', '66']]